# Product Detection
Shopee Code League 2020

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
for dirname, _, filenames in os.walk('./kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

./kaggle/input/order_brush_order.csv


# Loading data and do EDA (Exploratory Data Analysis)

In [3]:
import pandas as pd
import numpy as np
df_train = pd.read_csv('./kaggle/input/xr.csv')
df_train

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55
...,...,...,...,...
222745,31245479170194,110868129,193218997,2019-12-28 23:17:59
222746,31230440360374,91639906,3541807,2019-12-28 19:07:20
222747,31191471145838,29391773,135908070,2019-12-28 08:17:52
222748,31198471732255,61556313,27306783,2019-12-28 10:14:31


<img src= 'https://www.googleapis.com/download/storage/v1/b/kaggle-user-content/o/inbox%2F4342316%2F9c2161bc579fae87686285d953bdf56d%2Fdata%20description.png?generation=1591008771269892&alt=media'>

In [3]:
#check if there is any missing value
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1370 entries, 0 to 1369
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1370 non-null   int64 
 1   Attrition                 1370 non-null   object
 2   BusinessTravel            1370 non-null   object
 3   DailyRate                 1370 non-null   int64 
 4   Department                1370 non-null   object
 5   DistanceFromHome          1370 non-null   int64 
 6   Education                 1370 non-null   int64 
 7   EducationField            1370 non-null   object
 8   EmployeeCount             1370 non-null   int64 
 9   EmployeeNumber            1370 non-null   int64 
 10  EnvironmentSatisfaction   1370 non-null   int64 
 11  Gender                    1370 non-null   object
 12  HourlyRate                1370 non-null   int64 
 13  JobInvolvement            1370 non-null   int64 
 14  JobLevel                

In [5]:
#Check numerical data columns
print('There are %s numerical columns.'%(str(len(df_train.select_dtypes(include=['int64']).columns))))
df_train.select_dtypes(include=['int64']).columns

There are 26 numerical columns.


Index(['Age', 'DailyRate', 'DistanceFromHome', 'Education', 'EmployeeCount',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'MonthlyIncome',
       'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike',
       'PerformanceRating', 'RelationshipSatisfaction', 'StandardHours',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole',
       'YearsSinceLastPromotion', 'YearsWithCurrManager'],
      dtype='object')

In [6]:
#check categorical data columns 
print('There are %s numerical columns.'%(str(len(df_train.select_dtypes(include=['object']).columns))))
df_train.select_dtypes(include=['object']).columns

There are 9 numerical columns.


Index(['Attrition', 'BusinessTravel', 'Department', 'EducationField', 'Gender',
       'JobRole', 'MaritalStatus', 'Over18', 'OverTime'],
      dtype='object')

# Now we can divide all these 35 columns into 4 parts:
1. id column
2. target column
3. numerical columns
4. categorical columns

In [7]:
id_col = 'EmployeeNumber'

target_col = 'Attrition'

#25 numerical columns
numerical_cols = ['Age', 'DailyRate', 'DistanceFromHome', 'Education', 'EmployeeCount', 'EnvironmentSatisfaction', 
                'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'MonthlyIncome', 'MonthlyRate', 
                'NumCompaniesWorked', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 
                'StandardHours', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 
                'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager']

#8 categorical columns
categorical_cols = ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 
                 'Over18', 'OverTime']

In [8]:
#check 8 categorical columns
nunique_values = []
na_values = []
for col in categorical_cols:
    nunique_values.append(df_train[col].nunique())
    na_values.append(len(df_train.loc[df_train[col].isna()]))
df_train_stats_categorical = pd.DataFrame(list(zip(categorical_cols,nunique_values,na_values)),columns=['Column_Name','#Unique_Values','#Null_Value'])
df_train_stats_categorical

,Column_Name,#Unique_Values,#Null_Value
0,BusinessTravel,3,0
1,Department,3,0
2,EducationField,6,0
3,Gender,2,0
4,JobRole,9,0
5,MaritalStatus,3,0
6,Over18,1,0
7,OverTime,2,0


In [9]:
#check Null columns
df_train_stats_categorical[df_train_stats_categorical['#Null_Value']>0]

,Column_Name,#Unique_Values,#Null_Value


In [10]:
#check columns with only 1 value
df_train_stats_categorical[df_train_stats_categorical['#Unique_Values'] == 1]

,Column_Name,#Unique_Values,#Null_Value
6,Over18,1,0


The column 'Over18' only contains 1 unique value, therefore we could drop it.

In [11]:
#check 25 numerical columns
na_values = []
for col in numerical_cols:
    na_values.append(len(df_train.loc[df_train[col].isna()]))
df_train_stats_numerical = pd.DataFrame(list(zip(numerical_cols,na_values)),columns=['Column_Name','#Null_Value'])
df_train_stats_numerical

,Column_Name,#Null_Value
0,Age,0
1,DailyRate,0
2,DistanceFromHome,0
3,Education,0
4,EmployeeCount,0
5,EnvironmentSatisfaction,0
6,HourlyRate,0
7,JobInvolvement,0
8,JobLevel,0
9,JobSatisfaction,0


In [12]:
#check Null columns
df_train_stats_numerical[df_train_stats_numerical['#Null_Value']>0]

,Column_Name,#Null_Value


In [14]:
df_train_stats_numerical = df_train[numerical_cols].describe().T
df_train_stats_numerical

,count,mean,std,min,25%,50%,75%,max
Age,1370.0,36.927737,9.163392,18.0,30.0,36.0,43.00,60.0
DailyRate,1370.0,803.920438,402.834234,102.0,465.0,806.5,1156.00,1499.0
DistanceFromHome,1370.0,9.260584,8.135717,1.0,2.0,7.0,14.00,29.0
Education,1370.0,2.918978,1.022753,1.0,2.0,3.0,4.00,5.0
EmployeeCount,1370.0,1.000000,0.000000,1.0,1.0,1.0,1.00,1.0
EnvironmentSatisfaction,1370.0,2.728467,1.097507,1.0,2.0,3.0,4.00,4.0
HourlyRate,1370.0,65.845255,20.308706,30.0,48.0,66.0,83.00,100.0
JobInvolvement,1370.0,2.723358,0.719220,1.0,2.0,3.0,3.00,4.0
JobLevel,1370.0,2.067153,1.110354,1.0,1.0,2.0,3.00,5.0
JobSatisfaction,1370.0,2.727737,1.102307,1.0,2.0,3.0,4.00,4.0


In [15]:
#check columns with only 1 value
df_train_stats_numerical[df_train_stats_numerical['std']==0]

,count,mean,std,min,25%,50%,75%,max
EmployeeCount,1370.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
StandardHours,1370.0,80.0,0.0,80.0,80.0,80.0,80.0,80.0


Columns like 'EmployeeCount' and 'StandardHours' have 0 std, meaning all the values are the same, therefore we can drop them.

In [16]:
#check categorical columns
def explore_categorical_columns(df,categorical_cols):
    nunique_values = []
    na_values = []
    for col in categorical_cols:
        nunique_values.append(df[col].nunique())
        na_values.append(len(df.loc[df[col].isna()]))
    df_stats_categorical = pd.DataFrame(list(zip(categorical_cols,nunique_values,na_values)),columns=['Column_Name','#Unique_Values','#Null_Value'])
    df_null = df_stats_categorical[df_stats_categorical['#Null_Value']>0]
    df_unique_value = df_stats_categorical[df_stats_categorical['#Unique_Values'] == 1]
    if len(df_null) > 0:
        print('Columns with Null value: %s'%(str(df_null['Column_Name'])))
    else:
        print('There is no Null values in the categorical columns')
    if len(df_unique_value) > 0:
        print('Columns with only 1 unique value: %s'%(str(list(df_unique_value['Column_Name']))))
    else:
        print('All categorical columns have more than 1 value.')
        
#check numerical columns
def explore_numerical_columns(df,numerical_cols):
    na_values = []
    for col in numerical_cols:
        na_values.append(len(df.loc[df[col].isna()]))
    df_stats_numerical = pd.DataFrame(list(zip(numerical_cols,na_values)),columns=['Column_Name','#Null_Value'])
    df_null = df_stats_numerical[df_stats_numerical['#Null_Value']>0]
    df_stats = df[numerical_cols].describe().T
    df_unique_value = df_stats[df_stats['std']==0]
    
    if len(df_null) > 0:
        print('Columns with Null value: %s'%(str(df_null['Column_Name'])))
    else:
        print('There is no Null values in the numerical columns')
    
    if len(df_unique_value) > 0:
        print('Columns with only 1 unique value: %s'%(str(list(df_unique_value.index))))
    else:
        print('All numerical columns have more than 1 value. \n')

#check categorical columns
print('Categorical_Columns'.center(50,"*"))
explore_categorical_columns(df_train,categorical_cols)
print('Numerical_Columns'.center(50,"*"))
#check numerical columns
explore_numerical_columns(df_train,numerical_cols)

***************Categorical_Columns****************
There is no Null values in the categorical columns
Columns with only 1 unique value: ['Over18']
****************Numerical_Columns*****************
There is no Null values in the numerical columns
Columns with only 1 unique value: ['EmployeeCount', 'StandardHours']


- The column 'Over18' only contains 1 unique value, therefore we could drop it.
- Columns like 'EmployeeCount' and 'StandardHours' have 0 std, meaning all the values are the same, therefore we can drop them.

In [17]:
# Check target variable
df_train[target_col].unique()

array(['Yes', 'No'], dtype=object)

# Data Process

    # Scaling Numerical Data
<img src="https://miro.medium.com/max/1276/0*_apuT0HdrVYMUCh7">

In [75]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
def process_numerical_data(df,numerical_cols,drop_numerical_columns):
#     sacalar = MinMaxScaler()
    sacalar = StandardScaler() #if using StandardScaler
    scale_numerical_cols = list(set(numerical_cols)-set(drop_numerical_columns))
    df_numerical = sacalar.fit_transform(df[scale_numerical_cols])
    df_numerical = pd.DataFrame(df_numerical,columns=scale_numerical_cols)
    return df_numerical

drop_numerical_columns = ['EmployeeCount', 'StandardHours']
df_train_numerical = process_numerical_data(df_train,numerical_cols,drop_numerical_columns)
df_train_numerical

,YearsAtCompany,JobLevel,YearsSinceLastPromotion,JobInvolvement,HourlyRate,WorkLifeBalance,YearsInCurrentRole,TotalWorkingYears,TrainingTimesLastYear,EnvironmentSatisfaction,...,JobSatisfaction,StockOptionLevel,PercentSalaryHike,YearsWithCurrManager,Age,NumCompaniesWorked,DailyRate,MonthlyIncome,Education,PerformanceRating
0,-0.160912,-0.060501,-0.677130,0.384783,1.386845,-2.521504,-0.060481,-0.421351,-2.157001,-0.663990,...,1.154603,-0.938814,-1.154294,0.242164,0.444568,2.117138,0.740226,-0.112011,-0.898862,-0.423086
1,0.491304,-0.060501,-0.366873,-1.006120,-0.238667,0.334602,0.770876,-0.164452,0.149862,0.247499,...,-0.660436,0.244420,2.133975,0.799284,1.317926,-0.679467,-1.303544,-0.295072,-1.876972,2.363586
2,-1.139235,-0.961444,-0.677130,-1.006120,1.288329,0.334602,-1.168956,-0.549800,0.149862,1.158988,...,0.247084,-0.938814,-0.058205,-1.150636,0.007889,1.318108,1.413205,-0.939923,-0.898862,-0.423086
3,0.165196,-0.961444,0.253641,0.384783,-0.484957,0.334602,0.770876,-0.421351,0.149862,1.158988,...,0.247084,-0.938814,-1.154294,-1.150636,-0.428790,-0.679467,1.460388,-0.766195,1.057359,-0.423086
4,-0.813127,-0.961444,-0.056616,0.384783,-1.273084,0.334602,-0.614718,-0.678249,0.149862,-1.575478,...,-0.660436,0.244420,-0.880272,-0.593516,-1.083809,2.516653,-0.528749,-0.647619,-1.876972,-0.423086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1365,-0.323965,-0.060501,-0.677130,1.775686,-1.223826,0.334602,-0.614718,0.734692,0.149862,0.247499,...,1.154603,0.244420,0.489840,-0.314956,-0.101281,0.519078,0.198863,-0.837892,-0.898862,-0.423086
1366,0.002142,0.840441,-0.366873,-1.006120,-1.174568,0.334602,0.770876,-0.292902,1.687771,1.158988,...,-1.567955,0.244420,-0.058205,0.799284,0.226228,0.519078,-0.474116,0.736052,-1.876972,-0.423086
1367,-0.160912,-0.060501,-0.677130,1.775686,1.042039,0.334602,-0.614718,-0.678249,-2.157001,-0.663990,...,-0.660436,0.244420,1.311907,-0.314956,-1.083809,-0.679467,-1.611475,-0.080405,0.079248,2.363586
1368,0.328250,-0.060501,-0.677130,-1.006120,-0.140151,-1.093451,0.493757,0.734692,0.149862,1.158988,...,-0.660436,-0.938814,-0.332227,1.077845,1.317926,-0.279952,0.544044,-0.239921,0.079248,-0.423086


    # Convert categorical columns into numerical columns
<img src="https://i.imgur.com/mtimFxh.png">
    

In [76]:
df_train[categorical_cols]

,BusinessTravel,Department,EducationField,Gender,JobRole,MaritalStatus,Over18,OverTime
0,Travel_Rarely,Sales,Life Sciences,Female,Sales Executive,Single,Y,Yes
1,Travel_Frequently,Research & Development,Life Sciences,Male,Research Scientist,Married,Y,No
2,Travel_Rarely,Research & Development,Other,Male,Laboratory Technician,Single,Y,Yes
3,Travel_Frequently,Research & Development,Life Sciences,Female,Research Scientist,Married,Y,Yes
4,Travel_Rarely,Research & Development,Medical,Male,Laboratory Technician,Married,Y,No
...,...,...,...,...,...,...,...,...
1365,Travel_Frequently,Research & Development,Medical,Male,Laboratory Technician,Married,Y,No
1366,Travel_Rarely,Research & Development,Medical,Male,Healthcare Representative,Married,Y,No
1367,Travel_Rarely,Research & Development,Life Sciences,Male,Manufacturing Director,Married,Y,Yes
1368,Travel_Frequently,Sales,Medical,Male,Sales Executive,Married,Y,No


In [79]:
# ?pd.get_dummies

In [80]:
def process_categorical_data(df,categorical_cols,drop_categorical_columns):
    df_categorical = pd.get_dummies(df[[var for var in categorical_cols if var not in drop_categorical_columns]])
    return df_categorical

In [81]:
drop_categorical_columns = ['Over18']
df_train_categorical = process_categorical_data(df_train, categorical_cols, drop_categorical_columns)
df_train_categorical

,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,...,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_No,OverTime_Yes
0,0,0,1,0,0,1,0,1,0,0,...,0,0,0,1,0,0,0,1,0,1
1,0,1,0,0,1,0,0,1,0,0,...,0,0,1,0,0,0,1,0,1,0
2,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
3,0,1,0,0,1,0,0,1,0,0,...,0,0,1,0,0,0,1,0,0,1
4,0,0,1,0,1,0,0,0,0,1,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1365,0,1,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,1,0,1,0
1366,0,0,1,0,1,0,0,0,0,1,...,0,0,0,0,0,0,1,0,1,0
1367,0,0,1,0,1,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,1
1368,0,1,0,0,0,1,0,0,0,1,...,0,0,0,1,0,0,1,0,1,0


In [82]:
df_train_numerical.shape, df_train_categorical.shape

((1370, 23), (1370, 28))

In [83]:
df_train_features = pd.concat([df_train_numerical, df_train_categorical],axis=1)
df_train_features.shape

(1370, 51)

In [84]:
df_train_features

,YearsAtCompany,JobLevel,YearsSinceLastPromotion,JobInvolvement,HourlyRate,WorkLifeBalance,YearsInCurrentRole,TotalWorkingYears,TrainingTimesLastYear,EnvironmentSatisfaction,...,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_No,OverTime_Yes
0,-0.160912,-0.060501,-0.677130,0.384783,1.386845,-2.521504,-0.060481,-0.421351,-2.157001,-0.663990,...,0,0,0,1,0,0,0,1,0,1
1,0.491304,-0.060501,-0.366873,-1.006120,-0.238667,0.334602,0.770876,-0.164452,0.149862,0.247499,...,0,0,1,0,0,0,1,0,1,0
2,-1.139235,-0.961444,-0.677130,-1.006120,1.288329,0.334602,-1.168956,-0.549800,0.149862,1.158988,...,0,0,0,0,0,0,0,1,0,1
3,0.165196,-0.961444,0.253641,0.384783,-0.484957,0.334602,0.770876,-0.421351,0.149862,1.158988,...,0,0,1,0,0,0,1,0,0,1
4,-0.813127,-0.961444,-0.056616,0.384783,-1.273084,0.334602,-0.614718,-0.678249,0.149862,-1.575478,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1365,-0.323965,-0.060501,-0.677130,1.775686,-1.223826,0.334602,-0.614718,0.734692,0.149862,0.247499,...,0,0,0,0,0,0,1,0,1,0
1366,0.002142,0.840441,-0.366873,-1.006120,-1.174568,0.334602,0.770876,-0.292902,1.687771,1.158988,...,0,0,0,0,0,0,1,0,1,0
1367,-0.160912,-0.060501,-0.677130,1.775686,1.042039,0.334602,-0.614718,-0.678249,-2.157001,-0.663990,...,1,0,0,0,0,0,1,0,0,1
1368,0.328250,-0.060501,-0.677130,-1.006120,-0.140151,-1.093451,0.493757,0.734692,0.149862,1.158988,...,0,0,0,1,0,0,1,0,1,0


In [85]:
def process_label(df):
    target_col_dict = {'Yes': 1, 'No': 0}
    df_labels = df[target_col].map(target_col_dict).values
    return df_labels
df_train_labels = process_label(df_train)
df_train_labels.shape

(1370,)

In [86]:
df_train_labels

array([1, 0, 1, ..., 0, 0, 0])

In [87]:
# Convert array to Pandas DataFrame
df_train_labels = pd.DataFrame(df_train_labels)

# Concatenate features and labels
df_preprocessed_train = pd.concat([df_train_features, df_train_labels], axis=1)

In [88]:
df_preprocessed_train.head()

,YearsAtCompany,JobLevel,YearsSinceLastPromotion,JobInvolvement,HourlyRate,WorkLifeBalance,YearsInCurrentRole,TotalWorkingYears,TrainingTimesLastYear,EnvironmentSatisfaction,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_No,OverTime_Yes,0
0,-0.160912,-0.060501,-0.677130,0.384783,1.386845,-2.521504,-0.060481,-0.421351,-2.157001,-0.663990,...,0,0,1,0,0,0,1,0,1,1
1,0.491304,-0.060501,-0.366873,-1.006120,-0.238667,0.334602,0.770876,-0.164452,0.149862,0.247499,...,0,1,0,0,0,1,0,1,0,0
2,-1.139235,-0.961444,-0.677130,-1.006120,1.288329,0.334602,-1.168956,-0.549800,0.149862,1.158988,...,0,0,0,0,0,0,1,0,1,1
3,0.165196,-0.961444,0.253641,0.384783,-0.484957,0.334602,0.770876,-0.421351,0.149862,1.158988,...,0,1,0,0,0,1,0,0,1,0
4,-0.813127,-0.961444,-0.056616,0.384783,-1.273084,0.334602,-0.614718,-0.678249,0.149862,-1.575478,...,0,0,0,0,0,1,0,1,0,0


In [89]:
# Output preprocessed_training dataset
df_preprocessed_train.to_csv('HR-Employee-Attrition_train_preprocessed.csv', index=False, header=True)

In [90]:
df_preprocessed_train = pd.read_csv('HR-Employee-Attrition_train_preprocessed.csv')
# df_train_features = df_preprocessed_train.iloc[:, :-1]
# df_train_labels = df_preprocessed_train.iloc[:, -1]

In [91]:
df_preprocessed_train.head()

,YearsAtCompany,JobLevel,YearsSinceLastPromotion,JobInvolvement,HourlyRate,WorkLifeBalance,YearsInCurrentRole,TotalWorkingYears,TrainingTimesLastYear,EnvironmentSatisfaction,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_No,OverTime_Yes,0
0,-0.160912,-0.060501,-0.677130,0.384783,1.386845,-2.521504,-0.060481,-0.421351,-2.157001,-0.663990,...,0,0,1,0,0,0,1,0,1,1
1,0.491304,-0.060501,-0.366873,-1.006120,-0.238667,0.334602,0.770876,-0.164452,0.149862,0.247499,...,0,1,0,0,0,1,0,1,0,0
2,-1.139235,-0.961444,-0.677130,-1.006120,1.288329,0.334602,-1.168956,-0.549800,0.149862,1.158988,...,0,0,0,0,0,0,1,0,1,1
3,0.165196,-0.961444,0.253641,0.384783,-0.484957,0.334602,0.770876,-0.421351,0.149862,1.158988,...,0,1,0,0,0,1,0,0,1,0
4,-0.813127,-0.961444,-0.056616,0.384783,-1.273084,0.334602,-0.614718,-0.678249,0.149862,-1.575478,...,0,0,0,0,0,1,0,1,0,0


In [94]:
df_train_features = df_preprocessed_train.iloc[:, :-1]
df_train_labels = df_preprocessed_train.iloc[:, -1]

In [98]:
# split the data into train and test
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(df_train_features,df_train_labels,test_size=0.3,random_state=23)

# Set up and train the model

In [129]:
from sklearn.neural_network import MLPClassifier
input_size = df_train_features.shape[1]
NN = MLPClassifier(
    hidden_layer_sizes=(100, 100,),
    solver='adam',
    learning_rate='constant',
    learning_rate_init=0.001,
    max_iter=1000, 
    shuffle=True, 
    random_state=27,
    early_stopping=True)
NN.fit(train_x,train_y)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=True, epsilon=1e-08,
              hidden_layer_sizes=(100, 100), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=1000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=27, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [130]:
NN_predicted_y = NN.predict(test_x)

In [131]:
y_true, y_pred = test_y , NN_predicted_y
from sklearn.metrics import classification_report
print('Results on the test set:')
print(classification_report(y_true, y_pred))

Results on the test set:
              precision    recall  f1-score   support

           0       0.88      0.99      0.93       342
           1       0.81      0.30      0.44        69

    accuracy                           0.87       411
   macro avg       0.84      0.64      0.68       411
weighted avg       0.86      0.87      0.85       411



In [133]:
# from sklearn.metrics import accuracy_score
accuracy_score(test_y,NN_predicted_y)

0.8710462287104623

# Loading the test dataset and do the EDA

In [102]:
df_test = pd.read_csv('HR-Employee-Attrition_test.csv')
df_test

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,31,Travel_Rarely,1112,Sales,5,4,Life Sciences,1,1673,1,...,1,80,2,10,2,3,10,0,0,2
1,31,Travel_Rarely,1154,Sales,2,2,Life Sciences,1,1996,1,...,3,80,1,3,1,3,2,2,1,2
2,32,Travel_Frequently,379,Sales,5,2,Life Sciences,1,889,2,...,4,80,1,10,3,3,10,8,5,3
3,42,Travel_Rarely,188,Research & Development,29,3,Medical,1,1148,2,...,1,80,0,16,3,3,1,0,0,0
4,23,Travel_Rarely,373,Research & Development,1,2,Life Sciences,1,1270,4,...,4,80,1,1,2,3,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,32,Travel_Frequently,689,Sales,9,2,Medical,1,195,4,...,4,80,3,9,2,4,8,7,0,7
96,38,Travel_Rarely,330,Research & Development,17,1,Life Sciences,1,1088,3,...,1,80,1,20,4,2,19,9,1,9
97,44,Travel_Rarely,1037,Research & Development,1,3,Medical,1,2020,2,...,3,80,0,6,2,3,4,3,1,2
98,37,Travel_Rarely,671,Research & Development,19,3,Life Sciences,1,1631,3,...,1,80,3,9,2,2,4,3,0,2


In [103]:
df_test.head()

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,31,Travel_Rarely,1112,Sales,5,4,Life Sciences,1,1673,1,...,1,80,2,10,2,3,10,0,0,2
1,31,Travel_Rarely,1154,Sales,2,2,Life Sciences,1,1996,1,...,3,80,1,3,1,3,2,2,1,2
2,32,Travel_Frequently,379,Sales,5,2,Life Sciences,1,889,2,...,4,80,1,10,3,3,10,8,5,3
3,42,Travel_Rarely,188,Research & Development,29,3,Medical,1,1148,2,...,1,80,0,16,3,3,1,0,0,0
4,23,Travel_Rarely,373,Research & Development,1,2,Life Sciences,1,1270,4,...,4,80,1,1,2,3,1,0,0,1


In [104]:
print('Missing columns in test dataset(campared with train dataset): %s'%(str(set(df_train.columns)-set(df_test.columns))))

Missing columns in test dataset(campared with train dataset): {'Attrition'}


we can continue using the same 4 group column types:

In [105]:
id_col = 'EmployeeNumber'

numerical_cols = ['Age', 'DailyRate', 'DistanceFromHome', 'Education', 'EmployeeCount', 'EnvironmentSatisfaction', 'HourlyRate', 
                  'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 
                  'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel', 
                  'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 
                  'YearsSinceLastPromotion', 'YearsWithCurrManager']

categorical_cols = ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'Over18', 'OverTime']

In [106]:
#check 8 categorical columns
print('Categorical_Columns'.center(50,"*"))
explore_categorical_columns(df_test,categorical_cols)
print('Numerical_Columns'.center(50,"*"))
explore_numerical_columns(df_test,numerical_cols)

***************Categorical_Columns****************
There is no Null values in the categorical columns
Columns with only 1 unique value: ['Over18']
****************Numerical_Columns*****************
There is no Null values in the numerical columns
Columns with only 1 unique value: ['EmployeeCount', 'StandardHours']


In [107]:
drop_numerical_columns = ['EmployeeCount', 'StandardHours']
df_test_numerical = process_numerical_data(df_test,numerical_cols,drop_numerical_columns)
df_test_numerical

,YearsAtCompany,JobLevel,YearsSinceLastPromotion,JobInvolvement,HourlyRate,WorkLifeBalance,YearsInCurrentRole,TotalWorkingYears,TrainingTimesLastYear,EnvironmentSatisfaction,...,JobSatisfaction,StockOptionLevel,PercentSalaryHike,YearsWithCurrManager,Age,NumCompaniesWorked,DailyRate,MonthlyIncome,Education,PerformanceRating
0,0.450063,-0.018902,-0.707204,0.305304,0.023300,0.384111,-1.153607,-0.165344,-0.646162,-1.588374,...,1.134997,1.279204,-1.101914,-0.624588,-0.671425,-0.658969,0.798696,-0.169674,1.125260,-0.468521
1,-0.883456,-0.963982,-0.394282,0.305304,-0.607746,0.384111,-0.626846,-1.076689,-1.543610,-1.588374,...,0.234206,0.213201,1.012070,-0.624588,-0.671425,-1.073414,0.900604,-0.694589,-0.798261,-0.468521
2,0.450063,-0.018902,0.857406,0.305304,-0.898998,0.384111,0.953437,-0.165344,0.251285,-0.613911,...,-0.666586,0.213201,-0.309170,-0.315386,-0.557042,-0.658969,-0.979850,0.058682,-0.798261,-0.468521
3,-1.050146,-0.018902,-0.707204,-3.086964,-0.510662,0.384111,-1.153607,0.615809,0.251285,-0.613911,...,1.134997,-0.852803,1.012070,-1.242993,0.586782,0.584368,-1.443291,-0.432023,0.163499,-0.468521
4,-1.050146,-0.963982,-0.707204,0.305304,-0.947540,0.384111,-1.153607,-1.337073,-0.646162,1.335014,...,0.234206,0.213201,1.804814,-0.933790,-1.586484,-0.658969,-0.994409,-1.096392,-0.798261,2.134375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.116683,-0.018902,-0.707204,-3.086964,-1.530044,1.664479,0.690057,-0.295536,-0.646162,1.335014,...,1.134997,2.345208,0.483574,0.921422,-0.557042,-1.073414,-0.227668,-0.345735,-0.798261,-0.468521
96,1.950271,0.926178,-0.394282,-1.390830,-0.073784,-0.896258,1.216818,1.136577,1.148733,0.360551,...,0.234206,0.213201,0.747822,1.539827,0.129252,-1.073414,-1.098743,0.559628,-1.760022,-0.468521
97,-0.550076,-0.963982,-0.394282,0.305304,-1.190250,0.384111,-0.363465,-0.686112,-0.646162,-0.613911,...,1.134997,-0.852803,-0.837666,-0.624588,0.815547,1.413260,0.616716,-0.832082,0.163499,-0.468521
98,-0.550076,-0.018902,-0.707204,0.305304,0.897056,-0.896258,-0.363465,-0.295536,-0.646162,0.360551,...,0.234206,2.345208,0.483574,-0.624588,0.014870,0.169923,-0.271344,-0.106048,0.163499,-0.468521


In [108]:
# to check if there's any missing numerical columns in test dataset
set(df_train_numerical.columns)-set(df_test_numerical.columns)

set()

In [109]:
drop_categorical_columns = ['Over18']
df_test_categorical = process_categorical_data(df_test,categorical_cols,drop_categorical_columns)
df_test_categorical

,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,...,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_No,OverTime_Yes
0,0,0,1,0,0,1,0,1,0,0,...,0,0,0,1,0,0,1,0,1,0
1,0,0,1,0,0,1,0,1,0,0,...,0,0,0,0,1,0,1,0,1,0
2,0,1,0,0,0,1,0,1,0,0,...,0,0,0,1,0,0,1,0,1,0
3,0,0,1,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,1,0
4,0,0,1,0,1,0,0,1,0,0,...,0,0,1,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,1,0,0,0,1,0,0,0,1,...,0,0,0,1,0,1,0,0,1,0
96,0,0,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0
97,0,0,1,0,1,0,0,0,0,1,...,0,0,1,0,0,0,0,1,0,1
98,0,0,1,0,1,0,0,1,0,0,...,1,0,0,0,0,0,1,0,1,0


In [110]:
# to check if there's any missing categorical columns in test dataset
set(df_train_categorical.columns)-set(df_test_categorical.columns)

set()

In [111]:
df_test_features = pd.concat([df_test_numerical, df_test_categorical],axis=1)
df_test_features

,YearsAtCompany,JobLevel,YearsSinceLastPromotion,JobInvolvement,HourlyRate,WorkLifeBalance,YearsInCurrentRole,TotalWorkingYears,TrainingTimesLastYear,EnvironmentSatisfaction,...,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_No,OverTime_Yes
0,0.450063,-0.018902,-0.707204,0.305304,0.023300,0.384111,-1.153607,-0.165344,-0.646162,-1.588374,...,0,0,0,1,0,0,1,0,1,0
1,-0.883456,-0.963982,-0.394282,0.305304,-0.607746,0.384111,-0.626846,-1.076689,-1.543610,-1.588374,...,0,0,0,0,1,0,1,0,1,0
2,0.450063,-0.018902,0.857406,0.305304,-0.898998,0.384111,0.953437,-0.165344,0.251285,-0.613911,...,0,0,0,1,0,0,1,0,1,0
3,-1.050146,-0.018902,-0.707204,-3.086964,-0.510662,0.384111,-1.153607,0.615809,0.251285,-0.613911,...,0,0,0,0,0,0,0,1,1,0
4,-1.050146,-0.963982,-0.707204,0.305304,-0.947540,0.384111,-1.153607,-1.337073,-0.646162,1.335014,...,0,0,1,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.116683,-0.018902,-0.707204,-3.086964,-1.530044,1.664479,0.690057,-0.295536,-0.646162,1.335014,...,0,0,0,1,0,1,0,0,1,0
96,1.950271,0.926178,-0.394282,-1.390830,-0.073784,-0.896258,1.216818,1.136577,1.148733,0.360551,...,0,0,0,0,0,0,1,0,1,0
97,-0.550076,-0.963982,-0.394282,0.305304,-1.190250,0.384111,-0.363465,-0.686112,-0.646162,-0.613911,...,0,0,1,0,0,0,0,1,0,1
98,-0.550076,-0.018902,-0.707204,0.305304,0.897056,-0.896258,-0.363465,-0.295536,-0.646162,0.360551,...,1,0,0,0,0,0,1,0,1,0


In [112]:
df_test_features.to_csv('HR-Employee-Attrition_test_preprocessed.csv', index=False, header=True)

In [113]:
df_test_features = pd.read_csv('HR-Employee-Attrition_test_preprocessed.csv')

# Make prediction on test dataset

In [134]:
predicted_test = NN.predict(df_test_features)
predicted_test

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])

In [135]:
answer = df_test[['EmployeeNumber']]
answer['Attrition'] = ['Yes' if i == 1 else 'No' for i in predicted_test]
answer

/Users/haiho/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,EmployeeNumber,Attrition
0,1673,No
1,1996,No
2,889,No
3,1148,No
4,1270,No
...,...,...
95,195,No
96,1088,No
97,2020,No
98,1631,No


In [116]:
answer.to_csv('submission.csv',index=False)

# GridSearch to find the optimal parameter setting
Most used parameter

1）The ith element represents the number of neurons in the ith hidden layer.
    example: ``hidden_layer_sizes``=(10)
    
2）activation function
    example: ``activation``="relu"

3）The solver for weight optimization. {'lbfgs', 'sgd', 'adam'}
    example: ``solver``='adam'

4）L2 penalty (regularization term) parameter
    example: ``alpha``=0.0001

5）Size of minibatches for stochastic optimizers.
    If the solver is 'lbfgs', the classifier will not use minibatch.
    When set to 'auto', 'batch_size=min(200, n_samples)'
    example: ``batch_size``='auto'

6）Learning rate schedule for weight updates.
    example: ``learning_rate``="constant"

7）The initial learning rate used. It controls the step-size in updating the weights. Only used when solver='sgd' or 'adam'.
    example: ``learning_rate_init``=0.001

8）Maximum number of iterations. The solver iterates until convergence
    example: ``max_iter``=200

9）Tolerance for the optimization. When the loss or score is not improving
    by at least ``tol`` for ``n_iter_no_change`` consecutive iterations,
    unless ``learning_rate`` is set to 'adaptive', convergence is
    considered to be reached and training stops.
    example: ``tol``=1e-4

In [117]:
%%time
import time
start_time = time.time()
NN_new = MLPClassifier(max_iter=1000)
parameter_space = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (50,100,50,25), (input_size, 2*input_size, input_size)],
    'activation': ['tanh', 'relu','logistic'],
    'solver': ['adam', 'lbfgs', 'sgd'],
    'alpha': [0.001, 0.01],
    'learning_rate': ['constant','adaptive'],
}
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(NN_new, parameter_space, n_jobs=-1, cv=10)
clf.fit(train_x, train_y)
print('Time taken for training the model: '+ str(time.time() - start_time))

Time taken for training the model: 845.0878069400787
CPU times: user 30.4 s, sys: 690 ms, total: 31.1 s
Wall time: 14min 5s


/Users/haiho/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [118]:
# Best paramete set
print('Best parameters found:\n', clf.best_params_)

Best parameters found:
 {'activation': 'relu', 'alpha': 0.001, 'hidden_layer_sizes': (50, 100, 50), 'learning_rate': 'constant', 'solver': 'sgd'}


In [119]:
# All results
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

0.844 (+/-0.057) for {'activation': 'tanh', 'alpha': 0.001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'constant', 'solver': 'adam'}
0.849 (+/-0.045) for {'activation': 'tanh', 'alpha': 0.001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'constant', 'solver': 'lbfgs'}
0.862 (+/-0.045) for {'activation': 'tanh', 'alpha': 0.001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'constant', 'solver': 'sgd'}
0.836 (+/-0.051) for {'activation': 'tanh', 'alpha': 0.001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'adaptive', 'solver': 'adam'}
0.835 (+/-0.051) for {'activation': 'tanh', 'alpha': 0.001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'adaptive', 'solver': 'lbfgs'}
0.868 (+/-0.049) for {'activation': 'tanh', 'alpha': 0.001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'adaptive', 'solver': 'sgd'}
0.842 (+/-0.056) for {'activation': 'tanh', 'alpha': 0.001, 'hidden_layer_sizes': (50, 100, 50), 'learning_rate': 'constant', 'solver': 'adam'

In [120]:
y_true, y_pred = test_y , clf.predict(test_x)
from sklearn.metrics import classification_report
print('Results on the test set:')
print(classification_report(y_true, y_pred))

Results on the test set:
              precision    recall  f1-score   support

           0       0.91      0.93      0.92       342
           1       0.62      0.54      0.57        69

    accuracy                           0.87       411
   macro avg       0.76      0.73      0.75       411
weighted avg       0.86      0.87      0.86       411



In [121]:
accuracy_score(y_true, y_pred)

0.8661800486618005

In [122]:
test_y_pred = clf.predict(df_test_features)

In [123]:
df_result = pd.DataFrame.from_dict(dict({'EmployeeNumber':list(df_test['EmployeeNumber']),
                                         'Attrition':['Yes' if i == 1 else 'No' for i in test_y_pred]}))
df_result

,EmployeeNumber,Attrition
0,1673,No
1,1996,No
2,889,No
3,1148,No
4,1270,No
...,...,...
95,195,No
96,1088,No
97,2020,Yes
98,1631,No


In [124]:
# difference predictions between 2 models
temp = pd.merge(df_result,answer,on='EmployeeNumber',how='inner')
temp.loc[~(temp['Attrition_x']==temp['Attrition_y'])]

,EmployeeNumber,Attrition_x,Attrition_y
3,1148,No,Yes
9,120,Yes,No
20,1496,No,Yes
21,556,Yes,No
61,1173,No,Yes
64,466,Yes,No
70,475,Yes,No
81,1594,No,Yes


# Apply the best parameter setting

In [125]:
answer = df_result[['EmployeeNumber']]
answer['Attrition'] = ['Yes' if i == 1 else 'No' for i in test_y_pred]
answer

,EmployeeNumber,Attrition
0,1673,No
1,1996,No
2,889,No
3,1148,No
4,1270,No
...,...,...
95,195,No
96,1088,No
97,2020,Yes
98,1631,No


In [126]:
answer.to_csv('submission.csv',index=False)

# Random Forest

In [138]:
train_number_features = df_train_features.shape[1]
train_number_instances = df_train_features.shape[0]

In [156]:
from sklearn.ensemble import RandomForestClassifier

# train_x, test_x, train_y, test_y 
rf_clf = RandomForestClassifier(max_features='auto', max_depth=None, random_state=10)
rf_clf.fit(train_x, train_y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=10, verbose=0,
                       warm_start=False)

In [157]:
y_true, y_pred = test_y , rf_clf.predict(test_x)
from sklearn.metrics import classification_report
print('Results on the test set:')
print(classification_report(y_true, y_pred))

Results on the test set:
              precision    recall  f1-score   support

           0       0.86      0.98      0.92       342
           1       0.67      0.20      0.31        69

    accuracy                           0.85       411
   macro avg       0.76      0.59      0.61       411
weighted avg       0.83      0.85      0.81       411



In [144]:
test_y_pred = rf_clf.predict(df_test_features)

In [145]:
df_result = pd.DataFrame.from_dict(dict({'EmployeeNumber':list(df_test['EmployeeNumber']),
                                         'Attrition':['Yes' if i == 1 else 'No' for i in test_y_pred]}))
df_result

,EmployeeNumber,Attrition
0,1673,No
1,1996,No
2,889,No
3,1148,No
4,1270,No
...,...,...
95,195,No
96,1088,No
97,2020,No
98,1631,No


# AdaBoost

In [174]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomTreesEmbedding
ad_clf = AdaBoostClassifier(base_estimator=RandomForestClassifier(max_features='auto', max_depth=None, random_state=10), n_estimators=50, random_state=10, learning_rate=0.1)
ad_clf.fit(train_x, train_y)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=RandomForestClassifier(bootstrap=True,
                                                         ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='gini',
                                                         max_depth=None,
                                                         max_features='auto',
                                                         max_leaf_nodes=None,
                                                         max_samples=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                       

In [165]:
y_true, y_pred = test_y , ad_clf.predict(test_x)
from sklearn.metrics import classification_report
print('Results on the test set:')
print(classification_report(y_true, y_pred))

Results on the test set:
              precision    recall  f1-score   support

           0       0.86      0.99      0.92       342
           1       0.75      0.22      0.34        69

    accuracy                           0.86       411
   macro avg       0.81      0.60      0.63       411
weighted avg       0.84      0.86      0.82       411



In [168]:
test_y_pred = ad_clf.predict(df_test_features)
df_result = pd.DataFrame.from_dict(dict({'EmployeeNumber':list(df_test['EmployeeNumber']),
                                         'Attrition':['Yes' if i == 1 else 'No' for i in test_y_pred]}))
df_result

,EmployeeNumber,Attrition
0,1673,No
1,1996,No
2,889,No
3,1148,No
4,1270,No
...,...,...
95,195,No
96,1088,No
97,2020,No
98,1631,No


In [169]:
df_result.to_csv('submission.csv',index=False)

# Bagging

In [189]:
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC

bg_clf = BaggingClassifier(base_estimator=SVC(), n_estimators=20, random_state=10, max_features=35)
bg_clf.fit(train_x, train_y)

BaggingClassifier(base_estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                                     class_weight=None, coef0=0.0,
                                     decision_function_shape='ovr', degree=3,
                                     gamma='scale', kernel='rbf', max_iter=-1,
                                     probability=False, random_state=None,
                                     shrinking=True, tol=0.001, verbose=False),
                  bootstrap=True, bootstrap_features=False, max_features=35,
                  max_samples=1.0, n_estimators=20, n_jobs=None,
                  oob_score=False, random_state=10, verbose=0,
                  warm_start=False)

In [190]:
y_true, y_pred = test_y , bg_clf.predict(test_x)
from sklearn.metrics import classification_report
print('Results on the test set:')
print(classification_report(y_true, y_pred))

Results on the test set:
              precision    recall  f1-score   support

           0       0.85      1.00      0.92       342
           1       1.00      0.14      0.25        69

    accuracy                           0.86       411
   macro avg       0.93      0.57      0.59       411
weighted avg       0.88      0.86      0.81       411



In [191]:
test_y_pred = bg_clf.predict(df_test_features)
df_result = pd.DataFrame.from_dict(dict({'EmployeeNumber':list(df_test['EmployeeNumber']),
                                         'Attrition':['Yes' if i == 1 else 'No' for i in test_y_pred]}))
df_result

,EmployeeNumber,Attrition
0,1673,No
1,1996,No
2,889,No
3,1148,No
4,1270,No
...,...,...
95,195,No
96,1088,No
97,2020,No
98,1631,No


In [192]:
df_result.to_csv('submission.csv',index=False)